In [ ]:
# 토치텍스트가 제공하는 기능들은 다음과 같습니다.

# 파일 로드하기(File Loading) : 다양한 포맷의 코퍼스를 로드합니다.
# 토큰화(Tokenization) : 문장을 단어 단위로 분리해줍니다.
# 단어 집합(Vocab) : 단어 집합을 만듭니다.
# 정수 인코딩(Integer encoding) : 전체 코퍼스의 단어들을 각각의 고유한 정수로 맵핑합니다.
# 단어 벡터(Word Vector) : 단어 집합의 단어들에 고유한 임베딩 벡터를 만들어줍니다. 랜덤값으로 초기화한 값일 수도 있고, 사전 훈련된 임베딩 벡터들을 로드할 수도 있습니다.
# 배치화(Batching) : 훈련 샘플들의 배치를 만들어줍니다. 이 과정에서 패딩 작업(Padding)도 이루어집니다.

In [ ]:
!pip install torchtext

In [ ]:
import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

In [ ]:
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
df.head() # 상위 5개만 출력, Class는 2개 긍정(=1), 부정(=0)

In [ ]:
print("전체 샘플 개수 : {}".format(len(df)))

In [ ]:
train_df = df[:25000]
test_df = df[25000:]

In [ ]:
train_df.to_csv("train_data.csv", index=False) # 파일 저장을 따로 한다
test_df.to_csv("test_data.csv", index=False)

In [ ]:
# torchtext 에는 필드 도구를 제공함

from torchtext import data

TEXT = data.Field(sequential = True,
                  use_vocab = True,
                  tokenize = str.split,
                  batch_first = True,
                  fix_length = 20
                  )

LABEL = data.Field(sequential = False,
                   use_vocab = False,
                   batch_first = False,
                   is_target = True)


# 각 인자가 의미하는 바는 다음과 같습니다.

# sequential : 시퀀스 데이터 여부. (True가 기본값)
# use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
# tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
# lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
# batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
# is_target : 레이블 데이터 여부. (False가 기본값)
# fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.

In [ ]:
from torchtext.data import TabularDataset # 데이터를 불러오면서 필드에서 정의했던 토큰화 방법을 이용한다

train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

In [ ]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

In [ ]:
print(vars(train_data[0]))

In [ ]:
TEXT.build_vocab(train_data, min_freq = 10, max_size = 10000)

# min_freq : 단어 집합에 추가 시 단어의 최소 등장 빈도 조건을 추가.
# max_size : 단어 집합의 최대 크기를 지정.

In [ ]:
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

# 단어 집합의 크기를 10,000개로 제한하였지만 실제 생성된 단어 집합의 크기는
# 0번 단어부터 10,001번 단어까지 총 10,002개입니다.
# 이는 토치텍스트가 임의로 특별 토큰인 <unk>와 <pad>를 추가하였기 때문입니다

In [ ]:
print(TEXT.vocab.stoi)

In [ ]:
# 데이터 로더 만들기 - > 미니 배치만큼 데이터를 로드하게 만들어주는 역할을 한다

from torchtext.data import Iterator

batch_size = 5
train_loader = Iterator(dataset = train_data, batch_size = batch_size)
test_loader = Iterator(dataset = test_data, batch_size = batch_size)

In [ ]:
print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

# 25000개를 배치사이즈 5개로 했기 때문에 나오는 결과

In [ ]:
batch = next(iter(train_loader))

print(batch.text)

In [ ]:
batch = next(iter(train_loader))

print(batch.text[0])